### Primera aplicación basada en LLM

En esta lección vamos a integrar un LLM en una aplicación Python. Empezamos probando que podemos conversar un un modelo LLM. En estas y sucesivas lecciones, se usará el proveedor Google Gemini, cámbielo si prefiere usar otro.

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

MODEL = "gemini-2.0-flash"
openai = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta", api_key=api_key)

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "¿Cuánto son 2 + 2?"}]
)

print(response.choices[0].message.content)

2 + 2 son 4.



En la siguiente celda definimos la clase `Website` que permite hacer `scrapping` de una URL pasada como parámetro utilizando la librería `BeautifulSoup4`. Observe que lo que hace es obtener el `title` de la página y el texto. Para ello, elimina previamente el contenido de etiquetas irrelevantes con el método `BeautifulSoup.decompose()`.

In [2]:
import requests
from bs4 import BeautifulSoup

# A class to represent a Webpage
# Code from: https://github.com/ed-donner/llm_engineering 

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

En la siguiente celda, descargamos y mostramos el código de una página de Wikipedia.

In [16]:
# Versión en inglés de la página de Novak Djokovic
novak_url = "https://en.wikipedia.org/wiki/Novak_Djokovic"
novak_web = Website(url)
print(novak_web.title)
print(novak_web.text[350:400])

Novak Djokovic - Wikipedia
 editors
learn more
Contributions
Talk
Contents
mo


Creamos una función que cree el `prompt` de  `user` para resumir páginas de Wikipedia.

In [12]:
def user_prompt_for(website):
    user_prompt = f"Estás buscando una página con título {website.title}"
    user_prompt += "\nLos contenidos de este sitio web son los siguientes; \
                    por favor, proporciona un breve resumen de este sitio web en markdown. \
                    Si incluye noticias o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt

Creamos otra función añada el `prompt` de `system`:

In [49]:
system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
                    y proporciona un resumen breve, ignorando el texto que podría estar relacionado con la navegación. \
                    No añades ningún comentario inicial ni final. \
                    Respondes en markdown. Respondes en español."
    
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

Creamos la función que llama a la API para crear el resumen de la página:

In [19]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

Vemos que el resumen se hace en español a pesar de que la página está en inglés.

In [20]:
summarize(novak_url)

'Aquí está un resumen del perfil de Wikipedia de Novak Djokovic:\n\n*   **Resumen General:** Novak Djokovic es un tenista profesional serbio nacido el 22 de mayo de 1987. Es ampliamente considerado uno de los mejores tenistas de todos los tiempos.\n\n*   **Logros:** Djokovic ha sido clasificado como el número 1 del mundo por la Asociación de Tenistas Profesionales (ATP) durante un récord de 428 semanas y ha terminado como el número 1 de fin de año un récord de ocho veces. Tiene un récord de 24 títulos individuales importantes masculinos y 99 títulos individuales en total.\n\n*   **Carrera:** Djokovic se convirtió en profesional en 2003. Ganó su primer título importante en el Abierto de Australia en 2008. En 2011, alcanzó el número 1 por primera vez, ganando tres Majors y un récord entonces de cinco títulos de Masters, mientras que fue 10-1 contra Nadal y Federer.\n\n*   **Rivalidades:** Djokovic ha tenido rivalidades notables con Rafael Nadal, Roger Federer y Andy Murray.\n\n*   **Fuer

Para verlo mejor creamos una función que interprete el código Markdown:

In [70]:
from IPython.display import Markdown, display, update_display

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [40]:
display_summary(novak_url)

Novak Djokovic es un tenista profesional serbio nacido el 22 de mayo de 1987, considerado uno de los mejores de todos los tiempos. Ha mantenido el puesto número 1 del mundo por un récord de 428 semanas y ha terminado el año como número 1 en ocho ocasiones, también un récord. Djokovic ha ganado 24 títulos individuales masculinos de Grand Slam, incluyendo diez Abiertos de Australia. En total, ha ganado 99 títulos individuales, incluyendo 40 Masters y siete campeonatos de final de temporada, además de una medalla de oro olímpica. Es el único hombre en la historia del tenis en ser campeón defensor de los cuatro Grand Slams simultáneamente en tres superficies diferentes.

Djokovic comenzó su carrera profesional en 2003. En 2008, ganó su primer título de Grand Slam en el Abierto de Australia. En 2011, ascendió al número 1 por primera vez, ganando tres Grand Slams y un récord de cinco títulos Masters. En 2015, alcanzó un récord de 15 finales consecutivas y ganó 10 títulos grandes. En el Abierto de Francia de 2016, completó su primer Grand Slam de Carrera y un Grand Slam no calendario. Debido a su oposición a la vacuna COVID-19, se vio obligado a perderse muchos torneos en 2022. Representando a Serbia, Djokovic llevó al equipo nacional de tenis a su primer título de Copa Davis en 2010 y al título inaugural de la Copa ATP en 2020. Fuera de la competencia, Djokovic es filántropo y fundó la Fundación Novak Djokovic.

**¿Qué hemos conseguido?**

La capa gratuita de la API de Google Gemini no tiene acceso a Internet, y si le pedidos que nos resuma una página de Wikipedia, no puede hacerlo. Sin embargo, si le pasamos el texto de la página, sí que puede resumirlo. En este caso, hemos hecho un `scrapping` de la página y le hemos pasado el texto a la API. La API ha sido capaz de resumirlo y devolverlo en formato Markdown.

Podemos mejorar la clase Website para que incorpore información de los `links`que tenga la página que queremos resumir:

In [42]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

Vemos que hay muchos `links` que no son relevantes porque no tienen que ver con el instituto.

In [48]:
url_zayas = "https://site.educa.madrid.org/ies.mariadezayas.majadahonda/"
web_zayas = Website(url_zayas)
web_zayas.links[:10]

['#ht-content',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/',
 '#',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/quienes-somos/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/quienes-somos/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/informacion-centro/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-superior/',
 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-medio/']

Preparamos un `prompt` para que el modelo ignore los `links` irrelevantes. Utilizamos la técnica "one shot prompting" para instruir al modelo con un ejemplo. En este caso le decimos la estructura del JSON que queremos que genere.

In [54]:
link_system_prompt = "Se te proporciona una lista de enlaces encontrados en una página web. \
Eres capaz de decidir cuáles de los enlaces son más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, o una página de la Empresa, o páginas de Carreras/Empleos.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "página acerca de", "url": "https://url.completa/aquí/acerca"},
        {"type": "página de carreras", "url": "https://otra.url.completa/carreras"}
    ]
}
"""

Creamos el `prompt` de `user` para que el modelo genere el JSON:

In [53]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí tienes la lista de enlaces del sitio web de {website.url} - "
    user_prompt += "por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa, responde con la URL completa en formato JSON. \
                    No incluyas Términos de Servicio, Privacidad ni enlaces de correo electrónico. \
                    ni de fuera del sitio web \n"
    user_prompt += "Enlaces (algunos pueden ser enlaces relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


Usamos la IA para que decida los `links` más relevantes y los muestre en formato JSON:

In [51]:
import json

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [55]:
get_links(url_zayas)

{'links': [{'type': 'página acerca de',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/quienes-somos/'},
  {'type': 'información del centro',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/informacion-centro/'},
  {'type': 'oferta educativa',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/'},
  {'type': 'FP Grado Superior',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-superior/'},
  {'type': 'FP Grado Medio',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-medio/'},
  {'type': 'FP Básica',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-basica/'},
  {'type': 'Órganos de gestión',
   'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/organos-de-gestion/'},
  {'type': 'Dónde encontrarnos',
   'url': 

Creamos una función que añada los `links` al texto de la página:

In [56]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

Preparamos el `prompt` de `user` para que le pida a la IA que prepare un folleto con la información que reciba:

In [65]:
system_prompt = (
    "Eres un asistente que analiza el contenido de varias páginas relevantes de un sitio web de una empresa "
    "y crea un folleto corto, humorístico, entretenido y con chistes sobre la compañía para futuros clientes, "
    "inversores e interesados. Responde en markdown.\n\n"
    "Incluye detalles de la cultura de la empresa, los clientes y las carreras/empleos si tienes la información."
)

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás viendo una empresa llamada: {company_name}\n"
    user_prompt += "Aquí tienes el contenido de su página principal y otras páginas relevantes.\
                    usa esta información para crear un folleto breve de la empresa en markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

Creamos la función que llama a la API para crear el folleto y la probamos:

In [61]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [66]:
create_brochure("IES María de Zayas y Sotomayor", url_zayas)

Found links: {'links': [{'type': 'página acerca de', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/quienes-somos/'}, {'type': 'información del centro', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/informacion-centro/'}, {'type': 'oferta educativa', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/'}, {'type': 'FP Grado Superior', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-superior/'}, {'type': 'FP Grado Medio', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-medio/'}, {'type': 'FP Básica', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-basica/'}, {'type': 'órganos de gestión', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/organos-de-gestion/'}, {'type': 'dónde encontrarnos', 'url': 'https://site.educa.madri

Vale, aquí tienes un borrador de folleto humorístico sobre el IES María de Zayas y Sotomayor.

## ¿Cansado de la teoría? ¡Ven a IES María de Zayas y Sotomayor!

**(Porque aprender a hacer pan sabe mejor que aprender sobre el pan)**

[Imagen de un estudiante sonriendo mientras hornea algo delicioso]

**¿Quiénes somos?**

Somos el IES María de Zayas y Sotomayor, un centro público de Formación Profesional que te prepara para el mundo real... ¡y con sabor!  Olvida las clases aburridas y los libros polvorientos. Aquí aprenderás haciendo, creando y, sí, ¡probando!

**¿Qué ofrecemos?**

*   **FP Grado Superior, Medio y Básico:** Desde Desarrollo de Aplicaciones Multiplataforma y Web hasta Cocina y Restauración (¡con restaurante propio!). ¡Tenemos algo para ti!
*   **Aula de Emprendimiento:** ¿Tienes una idea millonaria? ¡Aquí te ayudamos a hacerla realidad! (Y si no, al menos aprenderás a hacer un buen café para vender).
*   **Aula de Gestión Turística:** Aprende a guiar turistas... ¡o a ser el turista que todos envidian!
*   **Proyecto Biodigestor del Zayas:** Convertimos residuos en energía... ¡y en chistes sobre ecología!
*   **Bolsa de Empleo:** ¡Encuentra trabajo antes de que te dé tiempo a echar de menos las clases!
*   **Erasmus:** ¡Vete de tapas por Europa mientras estudias!

**¿Por qué elegirnos?**

*   **Restaurante y Tienda:** Porque aprender con el estómago lleno es mucho más divertido.
*   **Aulas Especializadas:** Tenemos aulas para todo... ¡hasta para escapar de la rutina!
*   **Secretaría:** Resolveremos tus dudas... ¡o al menos lo intentaremos con una sonrisa!
*   **Eventos:** ¡Jornada de puertas abiertas el 15 de Mayo! ¡Ven a vernos y te prometemos que no te aburrirás!

**¿Quieres más info?**

*   Visita nuestra web: (la dirección web de IES María de Zayas y Sotomayor)
*   ¡Ven a conocernos en persona! (Pero trae hambre, por si acaso).

[Pequeña imagen divertida: un emoji guiñando un ojo con un gorro de chef.]

**IES María de Zayas y Sotomayor: Donde el aprendizaje es una aventura (y la comida está rica).**


Podemos generar la salida en tiempo real con el parámetro `stream = True`:

In [71]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [72]:
stream_brochure("IES María de Zayas y Sotomayor", url_zayas)

Found links: {'links': [{'type': 'página acerca de', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/quienes-somos/'}, {'type': 'información del centro', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/informacion-centro/'}, {'type': 'oferta educativa', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/'}, {'type': 'FP Grado Superior', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-superior/'}, {'type': 'FP Grado Medio', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-grado-medio/'}, {'type': 'FP Basica', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/oferta/fp-basica/'}, {'type': 'Órganos de Gestión', 'url': 'https://site.educa.madrid.org/ies.mariadezayas.majadahonda/index.php/el-centro/organos-de-gestion/'}, {'type': 'Dónde encontrarnos', 'url': 'https://site.educa.madri

Vale, aquí tienes un borrador del folleto humorístico del IES María de Zayas y Sotomayor.

# ¡IES María de Zayas y Sotomayor!

## ¿Quieres ser más listo que un libro de texto? ¡Nosotros te ayudamos!

**(Porque seamos sinceros, algunos libros de texto dan ganas de echarse una siesta... ¡y no precisamente reparadora!)**

[Imagen divertida de alguien durmiendo sobre un libro]

### ¿Quiénes somos?

Somos el IES María de Zayas y Sotomayor, ¡tu centro público de Formación Profesional! Y no, no nos hemos inventado el nombre con un generador aleatorio, ¡tenemos historia! (aunque no te vamos a aburrir con ella ahora mismo).

### ¿Qué ofrecemos?

Más ciclos formativos que excusas tienes para no ir al gimnasio. Desde Marketing y Publicidad hasta Cocina y Restauración, pasando por Desarrollo de Aplicaciones. ¡Tenemos algo para ti, a menos que quieras ser astronauta! (Para eso, mejor la NASA).

*   **FP Grado Superior:** Para los ambiciosos que quieren llegar a lo más alto.
*   **FP Grado Medio:** El punto intermedio perfecto para los que quieren aprender un oficio sin complicaciones.
*   **FP Grado Básico:** ¡El trampolín ideal para empezar con buen pie!

### ¡Nuestras Aulas son la leche!

¿Cansado de clases aburridas? ¡Nuestras aulas son como sacadas de una película! Tenemos:

*   **Aula de Emprendimiento:** Para que montes tu propio negocio y te hagas rico. ¡O al menos lo intentes!
*   **Aula de Gestión Turística:** ¡Conviértete en el próximo Indiana Jones del turismo! (Sin látigo, por favor).
*   **Restaurante y Tienda:** ¡Aquí se cocina el éxito! (Literalmente).
*   **Y muchas más...** ¡Ven a descubrirlas! (¡Tenemos más aulas que pelos tiene tu profe de mates!)

### ¿Buscas empleo? ¡Tenemos una Bolsa de Empleo!

Olvida buscar trabajo en los portales de empleo que te hacen sentir viejo. ¡En nuestra bolsa de empleo encontrarás oportunidades más frescas que un aguacate en su punto!

### ¿Erasmus? ¡Sí, por favor!

¿Quieres irte de fiesta por Europa mientras aprendes algo? ¡Con nuestro programa Erasmus es posible! (No nos hacemos responsables de resacas épicas).

### ¿Y la cafetería? ¡Importantísima!

¡Atención, amantes del café! Próximamente licitación de la cafetería del centro. ¡Si eres el próximo Starbucks, este es tu momento! (Si haces buen café, ¡ya tienes nuestro voto!).

### ¿Te lo estás pensando? ¡No lo dudes más!

Ven a nuestras jornadas de puertas abiertas el 15 de mayo. ¡Habrá tres turnos!

*   **Primer turno:** 10:00.
*   **Segundo turno:** 12:30.
*   **Tercer turno:** 17:00.

¡Te esperamos con los brazos abiertos y una sonrisa! (Y si traes galletas, ¡serás nuestro nuevo mejor amigo!)

### ¡IES María de Zayas y Sotomayor! ¡Aprender nunca fue tan divertido!

**(Bueno, quizás sí, pero no te vamos a mentir: ¡lo intentamos!)**

[Información de contacto del IES: dirección, teléfono, página web]
